Created on December 28th  2021 by Patrick Rotzetter

https://www.linkedin.com/in/rotzetter/

## Small experiment of document mining with various techniques Part 9

Let us use AWS built-in NTM algorithm for topic modeling

# Import libraries

In [2]:
# install NLTK and gensim if required
#%%sh
!pip3 -q install nltk gensim
!pip3 install texthero

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 24.2 MB 40.7 MB/s            
     |████████████████████████████████| 10.4 MB 60.6 MB/s            
     |████████████████████████████████| 235 

In [34]:
# Import require libraries
import numpy as np
import pandas as pd
import texthero as hero
import boto3

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


# Setup S3 parameters

In [52]:
#initialize some parameters depending where you are running the experiment, adapt the parameters to your AWS environment
bucket='mymltextarticles'
subfolder=''
region='us-east-1'

In [27]:
# let us list the files available for analysis in the S3 bucket
s3 = boto3.client('s3')
contents = s3.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
number_of_docs=0
for f in contents:
    number_of_docs=number_of_docs+1
    print(f['Key'])
    target_filename='./sampledocs/'+f['Key']
    s3.download_file(bucket, f['Key'], target_filename)

AI-bank-of-the-future-Can-banks-meet-the-AI-challenge-1.pdf.txt
Artificial Financial Intelligence.pdf.txt
Data machine the insurers using AI to reshape the industry Financial Times.pdf.txt
Digital-disruption-in-Insurance.pdf.txt
EPAM_Streamlining_the_Auto_Claims_Process_via_Integrated_IA.pdf.txt
Insurance-2030-The-impact-of-AI-on-the-future-of-insurance-F.pdf.txt
Issues_Paper_on_Increasing_Digitalisation_in_Insurance_and_its_Potential_Impact_on_Consumer_Outcomes.pdf.txt
Kaggle State of Machine Learning and Data Science 2020.pdf.txt
Module-1-Lecture-Slides.pdf.txt
Technology-and-innovation-in-the-insurance-sector.pdf.txt
WEF_Governance_of_Chatbots_in_Healthcare_2020.pdf.txt
ai-360-research.pdf.txt
ai-insurance.pdf.txt
fra-2020-artificial-intelligence_en.pdf.txt
sigma-5-2020-en.pdf.txt
sigma1_2020_en.pdf.txt


# Data preparation

In [28]:
#path of text files
path='./sampledocs/'

In [35]:
# let us scan the full directory, read PDF and PPT documents, clean them and process them with spacy

docName=[]
docType=[]
docText=[]
docNLP=[]
import glob
list_of_files = glob.glob(path+'*.txt')           # create the list of file
fileNames=[]
for file_name in list_of_files:
    f = open(file_name,'r')
    fileText=f.read()
    docName.append(file_name)
    docType.append('txt')
    docText.append(fileText)
fullDocs = pd.DataFrame({'Name':docName,'Type':docType,'Text':docText})
fullDocs['cleanText']=hero.clean(fullDocs['Text'])


In [36]:
 print ("Average length of text:" + str((np.mean(fullDocs['Text'].str.len()))))
 print ("Min length of text:" + str((np.min(fullDocs['Text'].str.len()))))
 print ("Max length of text:" + str((np.max(fullDocs['Text'].str.len()))))

Average length of text:90011.8125
Min length of text:9170
Max length of text:328295


In [37]:
fullDocs['text_word_count'] = fullDocs['Text'].apply(lambda x: len(x.strip().split()))  # word count
fullDocs['text_unique_words']=fullDocs['Text'].apply(lambda x:len(set(str(x).split())))  # number of unique words
fullDocs.head()

,Name,Type,Text,cleanText,text_word_count,text_unique_words
0,./sampledocs/Digital-disruption-in-Insurance.p...,txt,Digital disruption\nin insurance:\nCutting thr...,digital disruption insurance cutting noise con...,34485,7049
1,./sampledocs/Technology-and-innovation-in-the-...,txt,Technology and\ninnovation in the\ninsurance s...,technology innovation insurance sector technol...,16742,4228
2,./sampledocs/Data machine the insurers using A...,txt,Data machine: the insurers using AI to reshape...,data machine insurers using ai reshape industr...,1454,684
3,./sampledocs/sigma-5-2020-en.pdf.txt,txt,No 5 /2020\n\nMachine intelligence in\ninsuran...,machine intelligence insurance insights end en...,14478,4329
4,./sampledocs/ai-insurance.pdf.txt,txt,"AI in Insurance\nTop Use Cases, Challenges, an...",ai insurance top use cases challenges trends w...,6558,2297


In [38]:
fullDocs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               16 non-null     object
 1   Type               16 non-null     object
 2   Text               16 non-null     object
 3   cleanText          16 non-null     object
 4   text_word_count    16 non-null     int64 
 5   text_unique_words  16 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 896.0+ bytes


In [39]:
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
    
stop_words = stopwords.words('english')
wnl = WordNetLemmatizer()

def process_text(text):
    for p in string.punctuation:
        text = text.replace(p, '')
    text = ''.join([c for c in text if not c.isdigit()])
    text = text.lower().split()
    text = [w for w in text if not w in stop_words] 
    text = [wnl.lemmatize(w) for w in text]
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [40]:
%%time
fullDocs['cleanText'] = fullDocs['cleanText'].apply(process_text)

CPU times: user 2.7 s, sys: 78.3 ms, total: 2.78 s
Wall time: 2.79 s


In [41]:
fullDocs.head()

,Name,Type,Text,cleanText,text_word_count,text_unique_words
0,./sampledocs/Digital-disruption-in-Insurance.p...,txt,Digital disruption\nin insurance:\nCutting thr...,"[digital, disruption, insurance, cutting, nois...",34485,7049
1,./sampledocs/Technology-and-innovation-in-the-...,txt,Technology and\ninnovation in the\ninsurance s...,"[technology, innovation, insurance, sector, te...",16742,4228
2,./sampledocs/Data machine the insurers using A...,txt,Data machine: the insurers using AI to reshape...,"[data, machine, insurer, using, ai, reshape, i...",1454,684
3,./sampledocs/sigma-5-2020-en.pdf.txt,txt,No 5 /2020\n\nMachine intelligence in\ninsuran...,"[machine, intelligence, insurance, insight, en...",14478,4329
4,./sampledocs/ai-insurance.pdf.txt,txt,"AI in Insurance\nTop Use Cases, Challenges, an...","[ai, insurance, top, use, case, challenge, tre...",6558,2297


In [42]:
%%time

from gensim import corpora
dictionary = corpora.Dictionary(fullDocs['cleanText'])

CPU times: user 95.6 ms, sys: 3.12 ms, total: 98.7 ms
Wall time: 97.6 ms


In [43]:
print(dictionary)

Dictionary(10603 unique tokens: ['abandon', 'ability', 'able', 'abound', 'abraham']...)


In [44]:
dictionary.filter_extremes(keep_n=1024)
print(dictionary)

Dictionary(1024 unique tokens: ['academic', 'accelerate', 'accelerating', 'accept', 'accessed']...)


In [45]:
with open('vocab.txt', 'w') as f:
    for index in range(0,len(dictionary)):
        f.write(dictionary.get(index)+'\n')

In [46]:
%%time

fullDocs['tokens'] = fullDocs.apply(lambda row: dictionary.doc2bow(row['cleanText']), axis=1)

CPU times: user 49.6 ms, sys: 4.02 ms, total: 53.6 ms
Wall time: 56.6 ms


In [47]:
data = fullDocs.drop(['cleanText'], axis=1)
data = data.drop(['Name'], axis=1)
data = data.drop(['Type'], axis=1)
data = data.drop(['Text'], axis=1)
data = data.drop(['text_word_count'], axis=1)
data = data.drop(['text_unique_words'], axis=1)
data.head()

,tokens
0,"[(0, 1), (1, 7), (2, 1), (3, 1), (4, 2), (5, 5..."
1,"[(1, 2), (2, 1), (10, 1), (13, 2), (15, 2), (1..."
2,"[(14, 1), (16, 1), (58, 1), (60, 1), (77, 1), ..."
3,"[(0, 1), (1, 2), (3, 3), (7, 1), (8, 1), (9, 1..."
4,"[(2, 1), (6, 1), (8, 4), (9, 1), (12, 4), (14,..."


In [54]:
import sagemaker.amazon.common as smac
from scipy.sparse import lil_matrix

prefix = 'training'



In [49]:
def build_protobuf_dataset(data, dictionary):
    num_lines = data.shape[0]
    num_columns = len(dictionary)
    token_matrix = lil_matrix((num_lines, num_columns)).astype('float32')
    line = 0
    for _, row in data.iterrows():
        for token_id, token_count in row['tokens']:
            token_matrix[line, token_id] = token_count
        line+=1
        
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, token_matrix, None)
    return buf

In [50]:
def upload_protbuf_dataset(buf, bucket, prefix, key):
    obj = '{}/{}'.format(prefix, key)
    buf.seek(0)
    s3.upload_fileobj(buf, bucket, obj)
    path = 's3://{}/{}'.format(bucket,obj)
    return path

In [55]:
%%time
training_buf = build_protobuf_dataset(data, dictionary)
s3_training_path = upload_protbuf_dataset(training_buf, bucket, prefix, 'training/training.protobuf')
print(s3_training_path)

s3://mymltextarticles/training/training/training.protobuf
CPU times: user 75.7 ms, sys: 3.13 ms, total: 78.8 ms
Wall time: 139 ms


# Training

In [56]:
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_output)

s3://mymltextarticles/training/output/


In [61]:
from sagemaker.image_uris import retrieve
from sagemaker import get_execution_role

region = 'us-east-1'  
container = retrieve('lda', region)
print(container)

766337827248.dkr.ecr.us-east-1.amazonaws.com/lda:1


In [65]:
role = get_execution_role()
print(role)

lda = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1, 
    instance_type='ml.c4.2xlarge',
    output_path=s3_output)

arn:aws:iam::012086180905:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [66]:
lda.set_hyperparameters(
    num_topics=10, 
    feature_dim=len(dictionary), 
    mini_batch_size=1,
    alpha0=0.1)

In [67]:
lda.fit(inputs={'train': s3_training_path})

2022-01-01 13:57:20 Starting - Starting the training job...
2022-01-01 13:57:22 Starting - Launching requested ML instancesProfilerReport-1641045440: InProgress
...
2022-01-01 13:58:18 Starting - Preparing the instances for training.........
2022-01-01 13:59:45 Downloading - Downloading input data...
2022-01-01 14:00:12 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Using mxnet backend.
[01/01/2022 14:00:35 INFO 139762203416384] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'alpha0': u'1.0', u'max_restarts': u'10', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'allow_svd_init': u'true', u'epochs': u'1', u'tol': u'1e-8', u'_kvstore': u'local', u'max_iterations': u'1000'}
[01/01/2022 14:00:35 INFO 139762203416384] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'1024', u'mini_batch_size': u'1', u'num_topics': u'10

# Load trained model

In [12]:
import sagemaker
from sagemaker import get_execution_role
s3_output='s3://mymltextarticles/training/output/lda-2022-01-01-13-57-20-371/output/model.tar.gz'
role = get_execution_role()
lda=sagemaker.LDAModel(s3_output, role)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


In [13]:
lda_predictor = lda.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.large')

--------!

In [14]:
lda_predictor.delete_endpoint()

In [ ]:
!gunzip < output.tar.gz | tar -xv

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sea
import matplotlib as plt

In [ ]:
doc_topics=pd.read_csv('doc-topics.csv',lineterminator='\n')

In [ ]:
topic_terms=pd.read_csv('topic-terms.csv',lineterminator='\n')

In [ ]:
display(topic_terms)

In [ ]:
topic0_df=topic_terms[topic_terms['topic']==0]
plot=sea.barplot(x=topic0_df.term,y=topic0_df.weight,order=topic0_df.sort_values(by=['weight'], ascending=False).set_index('term').index)
plot.set_xticklabels(plot.get_xticklabels(),rotation=60)

In [ ]:
final_df = topic_terms.sort_values(by=['topic','weight'], ascending=False)
grid = sea.FacetGrid(final_df, col="topic", col_wrap=3,hue="topic",margin_titles=True, sharex=False)
grid_map=grid.map(sea.barplot,"term","weight",order=None);
for ax in grid_map.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(),rotation=60)
grid_map.tight_layout()

In [ ]:
topic_keywords = []
for i in range(number_of_topics):
    keywords = []
    df_sub= topic_terms[topic_terms['topic']==i].sort_values(by=['weight'], ascending=False)
    keywords = ", ".join(df_sub['term'])
    i=i+1
    topic_keywords.append(keywords)
topic_keywords_df= pd.DataFrame(topic_keywords)

In [ ]:
display(doc_topics)

In [ ]:
topics_df = pd.DataFrame()
for filename in filenames:
    sub_topics_df=doc_topics[doc_topics['docname']==filename].sort_values(by=['proportion'], ascending=False)
    row=sub_topics_df.iloc[0]
    topics_df = topics_df.append(pd.Series([row['docname'],int(row['topic']), round(row['proportion'],4)]), ignore_index=True)
topics_df.columns = ['Document name', 'Topic', 'Contribution']
topics_df

In [ ]:
#Top 3 Keywords for each Topic

topic_top3words = [(i,topic) for i, topics in enumerate(topic_keywords) for (j, topic) in enumerate(topics.split(',')) if j<3]
print(topic_top3words)

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)
df_top3words

In [ ]:
# Plot
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=120, sharey=True)

plt.figure(figsize=(15,10))
# Topic Distribution by Dominant Topics
ax=sea.histplot(data=topics_df,x=topics_df.Topic)
ax.set_xticks(range(topics_df.Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topic ' + str(x)+ '\n' + df_top3words.loc[df_top3words.topic_id==x, 'words'].values[0])
ax.xaxis.set_major_formatter(tick_formatter)
#ax.set_xticklabels(ax.get_xticklabels(),rotation=60)
ax.set_title('Number of Documents by Dominant Topic', fontdict=dict(size=10))
ax.set_ylabel('Number of Documents')
ax.set_ylim(0, 7)